In [1]:
import pandas as pd

df_train_classification = pd.read_parquet("data/train-classification.parquet")
trainable_usernames = df_train_classification["username"].tolist()

df_test = pd.read_csv("released_dataset/test-classification-round3.dat", header=None)
df_test.columns = ["username"]
test_usernames = df_test["username"].tolist()

print(f"Raw test usernames count: {len(test_usernames)}")


test_usernames_in_train = []
test_usernames_to_predict = []
# How many usernames in test set has its true labels in training set?
for test_username in test_usernames:
    if test_username in trainable_usernames:
        test_usernames_in_train.append(test_username)
    else:
        test_usernames_to_predict.append(test_username)

print(f"Usernames in test set that has correct labels in training data: {len(test_usernames_in_train)}")
print(f"Usernames in test set that has not correct labels in training data: {len(test_usernames_to_predict)}")



Raw test usernames count: 1000
Usernames in test set that has correct labels in training data: 266
Usernames in test set that has not correct labels in training data: 734


In [2]:
df_train = pd.read_parquet('data/training-dataset.parquet')

usernames_train_dataset = df_train['username'].tolist()

usernames_cant_predicted = []
for username in test_usernames_to_predict:
    if not username in usernames_train_dataset:
        usernames_cant_predicted.append(username)

print(f"There are {len(usernames_cant_predicted)} usernames that can't be predicted due to lack of data.")
print(f"These are {usernames_cant_predicted}")
df_train

# We can actually predict every username in the test set.

There are 0 usernames that can't be predicted due to lack of data.
These are []


,username,biography,bio_links,category_enum,label,posts
0,deparmedya,#mediaplanning #mediabuying #sosyalmedya,[http://www.deparmedya.com/],LOCAL,tech,"[Cumhuriyetimizin 100.yılı kutlu olsun♾️🇹🇷, Or..."
1,beyazyakaliyiz,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,[http://yinemipazartesi.wordpress.com/],PERSONAL_BLOG,None,[Bu diyaloğun yaşanmadığı bir online toplantı ...
2,kafesfirin,📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,[http://www.kafesfirin.com/],BRAND,food,[Bugün bir fincan köpüklü Türk kahvesiyle taçl...
3,vimerang,Dijital İletişim Yönetimi🎬info@vimerang.comq,[http://vimerang.com/],VIDEO_CREATOR,tech,[Saygı ve özlemle🖤\n\n#atatürk #10kasım #10kas...
4,totalenergies_istasyonlari,TotalEnergies İstasyonları resmi Instagram hes...,[https://bit.ly/45XDUDr],ENERGY_COMPANY,None,[Başöğretmenimiz Gazi Mustafa Kemal Atatürk’ün...
...,...,...,...,...,...,...
5410,herbisiatolye,🌱 Şifalı Bitkiler Atölyesi 🌾\n📲 WhatsApp Sipar...,[http://www.herbisi.com.tr/],HEALTH_BEAUTY,health and lifestyle,[Gurdjieff kutsal dansları ve öğretisi çalışma...
5411,nestleturkiye,"117 YILDIR YUVAMIZ TÜRKİYE! \n5 fabrikamız, 30...",[https://www.nestle.com.tr/hakkimizda],FOOD_COMPANY,None,[Aramızdan ayrılışının 85. yılında Gazi Mustaf...
5412,tariszeytinyagi,None,[http://www.tariszeytin.com.tr],ACTIVITY_GENERAL,None,"[DÜNYA DİYABET GÜNÜ\n\nŞeker hastalığı, vücudu..."
5413,iklim_ce,👇👇İklimce Sohbetler’le ilgili her şey,[https://linktr.ee/iklim_ce],PERSON,None,[Enerji ve Tabii Kaynaklar Bakanlığı’nın açıkl...


To do both the training and the test, we need embeddings of the usernames.

1. Usernames that are in the train set which has the correct category values. (To Train)
2. Useranames that are in the test set which do not have the correct category value. (To Test)

In [3]:
test_usernames_to_predict

['livapastanesi',
 'tusasshop',
 'etolyadigital',
 'tulugozlu',
 'krossbisiklet',
 'haribochamallows',
 'ozatashipyard',
 'yenisafak',
 'burcinterzioglu',
 'baselifeclub',
 'imuneksfarma',
 'deutz_fahr_turkey',
 'tezatsanat',
 'filtresizcom',
 'westchocolatemarina',
 'sebnemcapa',
 'rozetsepeti',
 'ececesmioglu',
 'gocaagonyali',
 'oztayteksofficial',
 'imtolstoyevski',
 'rustik.rus.restorani',
 'mehmetcikvakfi',
 'bruderturkiye',
 'brandingturkiye',
 'dogushyilmaz',
 'crmbelediyesi',
 'gcaturkiye',
 'resimlimagnet_com',
 'ilaydaakdogan',
 'karincaproduksiyon',
 'ilkergodencemobilya',
 'ogretmeninkaleminden_',
 'narkoyjournal',
 'kuzucu_mucevherat',
 'mmkmetalurji',
 'okanyanmazcom',
 'silopibld',
 'cubocomtr',
 'ahlatbld',
 'komunpub',
 'novax_gozluk_camlari',
 'tarkan',
 'sefkatpeker',
 'kore_kultur_merkezi',
 'gaiaandco',
 'bvbajans',
 'esila_kids',
 'halisunasyoncoffee',
 'ecovero.turkey',
 'karaman.bel.tr',
 'imoykukarayel',
 'celsuskitabevi',
 'keyifbebesi',
 'vadistanbul',
 'piv

In [4]:
train_usernames = df_train.dropna(subset=['label'])['username'].tolist()
train_usernames

['deparmedya',
 'kafesfirin',
 'vimerang',
 'mustafa_yalcinn38',
 'zorluenergysolutions',
 'billsfooddrink',
 'cpmyazilim',
 'fourestcalticak',
 'besiktasjksporokullari',
 'altindagbel',
 'orientturkiye',
 'altekyapi',
 'revoday',
 'sinankoc',
 'gedizbld',
 'diyetisyencereniseri',
 'civilimcom',
 'pastaciduzce',
 'sbbtv55',
 'inovagelinlik_kayseri',
 'ugurbilgin',
 'birselvardarlidemirmen',
 'isikkent_egitim_kampusu',
 'dr.yaseminsavas',
 'odda75_',
 'kravatistan',
 'sinemozler',
 'revoxlaboratories',
 'turktelekom',
 'rahmi_dogan',
 'byokuyanmasasandalye',
 'macrocentertr',
 'gezinomi',
 'duja_hotels',
 'muratercan0606',
 'bgst_tiyatro',
 'dogankabak',
 'elvankreatif',
 'fgikapi',
 'bugusto',
 'bakuslunalounge',
 'fitbas',
 'keymenilac',
 'askmoyra_en',
 'by',
 'expo2016antalya',
 'madamecocotr',
 'nenibrasserie',
 'bahcekadikoyrestaurant',
 'yildiz_teknopark',
 'cevaheer',
 'diyetisyengamzealtinay',
 'selvagida',
 'ozgeozgencc',
 'kaanlar',
 'selahattindonmez',
 'burulasulasim',
 'tc

In [5]:
all_usernames_to_embed = [*train_usernames, *test_usernames_to_predict]

df_username_biography = pd.read_parquet("data/username_biography.parquet")
df_username_caption = pd.read_parquet("data/username_caption.parquet")

df_uc = df_username_caption[df_username_caption["username"].isin(all_usernames_to_embed)]
df_ub = df_username_biography[df_username_biography["username"].isin(all_usernames_to_embed)]

df_uc.to_parquet('data/username_caption_2em.parquet')
df_ub.to_parquet('data/username_biography_2em.parquet')

